In [ ]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, chain, combinations
from scipy import stats
from IPython.display import display, HTML
%matplotlib inline

In [ ]:
inputFile = 'preprocessed_data.csv'
metaFile = 'meta.csv'

In [ ]:
df = pd.read_csv(inputFile)
del df['FileName']
del df['timestamp']
target = df.loc[:,'INDOOR'].to_numpy()
del df['INDOOR']

df_meta = pd.read_csv(metaFile)
del df_meta['INDOOR']

In [ ]:
import seaborn as sns
corr = df.corr()
sns.heatmap(corr)

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.to_numpy(), target, test_size=0.33) 
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
D = X_train.shape[1]
D

In [ ]:
# Now all the fun Tensorflow stuff
# Build the model

i = tf.keras.layers.Input(shape=(D,),name='Input')
x = tf.keras.layers.Dense(16,activation='sigmoid', name='Hidden')(i)
x = tf.keras.layers.Dense(1, activation='sigmoid', name='Classifier')(x)
model = tf.keras.models.Model(i, x)


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

r = model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=50)

print("Train score:", model.evaluate(X_train, y_train))
print("Test score:", model.evaluate(X_test, y_test))

In [ ]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
# Plot the accuracy too
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()

In [ ]:
model.summary()

In [ ]:
for z in zip(df_wide.columns.values,model.layers[5].get_weights()[0]):
  print(z)

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True,show_dtype=True, show_layer_names=True, expand_nested=True)

In [ ]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

In [ ]:
# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "IODetector model"
model_meta.description = "Detect if the user is indoor or outdoor."
model_meta.version = "v1"
model_meta.author = "Dario Lorenzoni, Niccolò Scatena"
model_meta.license = "MIT LICENSE"

In [ ]:
# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()

In [ ]:
input_meta.name = "feature_vector"
input_meta.description = "IODetection input vector"
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (_metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = df_meta['mean'].values.tolist()
input_normalization.options.std = df_meta['std'].values.tolist()
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = df_meta['max'].values.tolist()
input_stats.min = df_meta['min'].values.tolist()
input_meta.stats = input_stats

In [ ]:
# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "IODetection"
output_meta.description = "Indoor/Outdoor label."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (_metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.min = [0.0]
output_stats.max = [1.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename("iomap.txt")
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]

In [ ]:
# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

In [ ]:
populator = _metadata.MetadataPopulator.with_model_file(model_file)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files(["iomap.txt"])
populator.populate()

In [ ]:
#tf.saved_model.save(model, "saved_model_keras_dir")
"""
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=True
converter.experimental_new_converter =True
"""
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
displayer = _metadata.MetadataDisplayer.with_model_file('model.tflite')
export_json_file = "meta.json"
json_file = displayer.get_metadata_json()
with open(export_json_file, "w") as f:
  f.write(json_file)